>[Notebook 4: Sentiment Analysis of News](#scrollTo=Yv71FhcVKcmr)

>>[Part 1: Load Dataset](#scrollTo=Y3loqydbSOJz)

>>[Part 2: Apply Sentiment Analysis to Dataset](#scrollTo=cI3a-ym2eB1D)

>>[Part 3: Statistical Approach](#scrollTo=3NmwbIRVe2qN)



# Notebook 4: Sentiment Analysis of News



This notebook will contain analysis related to Sentiment Analysis of News

## Part 1: Load Dataset

In [1]:
cd /content/drive/MyDrive/CIND820

/content/drive/MyDrive/CIND820


In [2]:
# import required libraries
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report


import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

Generated dataframes are in generated_dfs directory under main directory. This dataframe was generated from the steps in Notebook 2

In [3]:
# load dataset
news_df = pd.read_csv('generated_dfs/news_df_v6.csv')

<ipython-input-3-88db05494c22>:2: DtypeWarning: Columns (0,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  news_df = pd.read_csv('generated_dfs/news_df_v6.csv')


In [4]:
news_df.head()

,article_id,source_id,source_name,author,title,description,url,url_to_image,published_at,content,...,bigrams,vader_sentiment,neg,neu,pos,compound,pos_tags,NER,DJIA_Open,DJIA_Close
0,94333,business-insider,Business Insider,Sponsor Post,Unlocking the future of the web,"From Chipotle to Roblox, web3 is enabling busi...",https://www.businessinsider.com/sc/web3-helps-...,https://i.insider.com/654374023cc84b4dfafa98cb...,2023-11-02 16:25:49,Adobe Stock\nHarnessing customer engagement an...,...,"[('Unlocking', 'the'), ('the', 'future'), ('fu...","{'neg': 0.026, 'neu': 0.816, 'pos': 0.158, 'co...",0.026,0.816,0.158,0.8225,"[('Unlocking', 'VBG'), ('future', 'JJ'), ('web...",(S\n Unlocking/VBG\n future/JJ\n web/JJ\n ...,37949.671875,37753.308594
1,94368,NaN,The Indian Express,Reuters,"LinkedIn hits 1 billion members, adds AI featu...",LinkedIn also introduced on Wednesday a button...,https://indianexpress.com/article/technology/s...,https://images.indianexpress.com/2023/11/Linke...,2023-11-02 03:48:44,"LinkedIn, the business-focused social network ...",...,"[('LinkedIn', 'hits'), ('hits', '1'), ('1', 'b...","{'neg': 0.012, 'neu': 0.886, 'pos': 0.101, 'co...",0.012,0.886,0.101,0.9733,"[('LinkedIn', 'NNP'), ('hits', 'VBZ'), ('1', '...",(S\n (ORGANIZATION LinkedIn/NNP)\n hits/VBZ\...,37949.671875,37753.308594
2,94370,NaN,Investor's Business Daily,Investor's Business Daily,"Moderna Beats Sales Forecasts, But Light Guida...",The company issued below-consensus sales views...,https://www.investors.com/news/technology/mode...,https://www.investors.com/wp-content/uploads/2...,2023-11-02 10:30:25,Moderna (MRNA) stock could take a hit Thursday...,...,"[('Moderna', 'Beats'), ('Beats', 'Sales'), ('S...","{'neg': 0.065, 'neu': 0.883, 'pos': 0.052, 'co...",0.065,0.883,0.052,-0.1901,"[('Moderna', 'NNP'), ('Beats', 'NNP'), ('Sales...",(S\n (PERSON Moderna/NNP)\n Beats/NNP\n Sal...,37949.671875,37753.308594
3,94331,abc-news,ABC News,ABC News,WATCH: Man rescued from crashed plane in the E...,A man was hoisted to safety after a small plan...,https://abcnews.go.com/US/video/man-rescued-cr...,https://i.abcnewsfe.com/a/dee71d57-ad91-4eec-8...,2023-11-02 10:19:28,<ul><li>Whats next for Russia? \n</li><li>What...,...,"[('WATCH', 'Man'), ('Man', 'rescued'), ('rescu...","{'neg': 0.047, 'neu': 0.84, 'pos': 0.112, 'com...",0.047,0.840,0.112,0.5484,"[('WATCH', 'NNP'), ('Man', 'NNP'), ('rescued',...",(S\n (ORGANIZATION WATCH/NNP)\n (PERSON Man/...,37949.671875,37753.308594
4,94332,abc-news,ABC News,ABC News,WATCH: Teen solves Rubik’s cube while skydiving,"Sam Sieracki, 17, broke the world record by so...",https://abcnews.go.com/GMA/Living/video/teen-s...,https://i.abcnewsfe.com/a/a4f53cb3-cbfe-4f01-a...,2023-11-02 12:16:02,<ul><li>Whats next for Russia? \n</li><li>What...,...,"[('WATCH', 'Teen'), ('Teen', 'solves'), ('solv...","{'neg': 0.109, 'neu': 0.808, 'pos': 0.083, 'co...",0.109,0.808,0.083,-0.3485,"[('WATCH', 'NNP'), ('Teen', 'NNP'), ('solves',...",(S\n (ORGANIZATION WATCH/NNP Teen/NNP)\n sol...,37949.671875,37753.308594


In [5]:
news_df.columns

Index(['article_id', 'source_id', 'source_name', 'author', 'title',
       'description', 'url', 'url_to_image', 'published_at', 'content',
       'category', 'full_content', 'article', 'title_sentiment', 'all_text',
       'cleaned_text', 'tickers', 'tokens', 'num_tokens',
       'tokens_without_stopwords', 'stemmed_tokens', 'lemmatized_tokens',
       'bow_vocab', 'unigrams', 'bigrams', 'vader_sentiment', 'neg', 'neu',
       'pos', 'compound', 'pos_tags', 'NER', 'DJIA_Open', 'DJIA_Close'],
      dtype='object')

## Part 2: Apply Sentiment Analysis to Dataset

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Define the threshold
threshold = 0
y = news_df['compound'].fillna(0)

# Convert continuous target variable to binary labels
y_binary = (y > threshold).astype(int)

# Prepare the data for machine learning algorithms
X = news_df['cleaned_text']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [7]:
# Initialize and train the Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)
nb_predictions = nb_classifier.predict(X_test_tfidf)
print("Naive Bayes Accuracy:", accuracy_score(y_test, nb_predictions))
print(classification_report(y_test, nb_predictions))

# Initialize and train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(X_train_tfidf, y_train)
rf_predictions = rf_classifier.predict(X_test_tfidf)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_predictions))
print(classification_report(y_test, rf_predictions))

# Initialize and train the Logistic Regression classifier
lr_classifier = LogisticRegression(max_iter=1000)
lr_classifier.fit(X_train_tfidf, y_train)
lr_predictions = lr_classifier.predict(X_test_tfidf)
print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_predictions))
print(classification_report(y_test, lr_predictions))

# Initialize and train the AdaBoost classifier
ab_classifier = AdaBoostClassifier(n_estimators=100)
ab_classifier.fit(X_train_tfidf, y_train)
ab_predictions = ab_classifier.predict(X_test_tfidf)
print("AdaBoost Accuracy:", accuracy_score(y_test, ab_predictions))
print(classification_report(y_test, ab_predictions))


Naive Bayes Accuracy: 0.8613973182780522
              precision    recall  f1-score   support

           0       0.81      0.61      0.70      1846
           1       0.87      0.95      0.91      5239

    accuracy                           0.86      7085
   macro avg       0.84      0.78      0.80      7085
weighted avg       0.86      0.86      0.85      7085

Random Forest Accuracy: 0.9112208892025406
              precision    recall  f1-score   support

           0       0.95      0.70      0.80      1846
           1       0.90      0.99      0.94      5239

    accuracy                           0.91      7085
   macro avg       0.93      0.84      0.87      7085
weighted avg       0.91      0.91      0.91      7085

Logistic Regression Accuracy: 0.903316866619619
              precision    recall  f1-score   support

           0       0.88      0.72      0.80      1846
           1       0.91      0.97      0.94      5239

    accuracy                           0.90      7

## Part 3:Possible Association with Stock Market Price

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Prepare the data for machine learning algorithms
X = news_df['cleaned_text']
y = news_df['DJIA_Close'].fillna(0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Random Forest Regression': RandomForestRegressor()
}

# Train and evaluate models
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f'\n{name} Results:')
    print(f'Mean Squared Error: {mse}')
    print(f'Root Mean Squared Error: {rmse}')
    print(f'Mean Absolute Error: {mae}')
    print(f'R-squared: {r2}\n')


Training Linear Regression...

Linear Regression Results:
Mean Squared Error: 0.24481128909218303
Root Mean Squared Error: 0.4947840833052161
Mean Absolute Error: 0.17682357502018473
R-squared: 0.9999999974194661

Training Ridge Regression...

Ridge Regression Results:
Mean Squared Error: 25175.671646892224
Root Mean Squared Error: 158.6684330511026
Mean Absolute Error: 59.54692282845346
R-squared: 0.9997346255025521

Training Lasso Regression...


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 519776990.45317936, tolerance: 251046884.2344256
  model = cd_fast.sparse_enet_coordinate_descent(



Lasso Regression Results:
Mean Squared Error: 13416.480194049083
Root Mean Squared Error: 115.82953075122545
Mean Absolute Error: 10.886683384214882
R-squared: 0.9998585780852661

Training Random Forest Regression...

Random Forest Regression Results:
Mean Squared Error: 7181.884186162952
Root Mean Squared Error: 84.74599805396684
Mean Absolute Error: 1.4387287678634435
R-squared: 0.9999242964027588

